# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [6]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City"
)
# Display the map
# YOUR CODE HERE
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_condition_df=city_data_df.loc[(city_data_df["Max Temp"]<26) & (city_data_df["Max Temp"]>17) & (city_data_df["Cloudiness"]<50) & (city_data_df["Humidity"]<55)]

# Drop any rows with null values
# YOUR CODE HERE
city_clean_data_df=ideal_weather_condition_df.dropna()
# Display sample data
# YOUR CODE HERE
city_clean_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
106,106,palencia,42.4167,-4.5000,24.40,30,40,8.66,ES,1666108219
229,229,san cristobal,7.7669,-72.2250,22.10,45,46,0.57,VE,1666108372
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102
258,258,comodoro rivadavia,-45.8667,-67.5000,25.92,24,0,7.20,AR,1666108389
296,296,jalu,29.0331,21.5482,22.33,37,25,3.89,LY,1666108413
305,305,kingman,35.1894,-114.0530,20.07,49,5,4.02,US,1666108417
328,328,viseu,40.6610,-7.9097,22.70,39,34,6.13,PT,1666108429


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df=city_clean_data_df[["City","Country","Lat","Lng","Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"]=""
# Display sample data
# YOUR CODE HERE
hotel_df

C:\Users\nirav\AppData\Local\Temp\ipykernel_17512\2375554443.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
63,banda,IN,25.4833,80.3333,52,
73,nalut,LY,30.3333,10.8500,27,
106,palencia,ES,42.4167,-4.5000,30,
229,san cristobal,VE,7.7669,-72.2250,45,
239,rawson,AR,-43.3002,-65.1023,25,
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,
296,jalu,LY,29.0331,21.5482,37,
305,kingman,US,35.1894,-114.0530,49,
328,viseu,PT,40.6610,-7.9097,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
limit=20
categories = "accommodation.hotel"

params = {"categories":categories,"limit":limit,"apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude=row["Lng"]
    latitude=row["Lat"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
banda - nearest hotel: #acnindiafy21
nalut - nearest hotel: No hotel found
palencia - nearest hotel: Hotel Área de Servicio Los Chopos
san cristobal - nearest hotel: Terra Sur
rawson - nearest hotel: Hotel Deportivo
comodoro rivadavia - nearest hotel: No hotel found
jalu - nearest hotel: فندق جالو السياحي
kingman - nearest hotel: Home2 Suites by Hilton Kingman
viseu - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
73,nalut,LY,30.3333,10.8500,27,No hotel found
106,palencia,ES,42.4167,-4.5000,30,Hotel Área de Servicio Los Chopos
229,san cristobal,VE,7.7669,-72.2250,45,Terra Sur
239,rawson,AR,-43.3002,-65.1023,25,Hotel Deportivo
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,No hotel found
296,jalu,LY,29.0331,21.5482,37,فندق جالو السياحي
305,kingman,US,35.1894,-114.0530,49,Home2 Suites by Hilton Kingman
328,viseu,PT,40.6610,-7.9097,39,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols=['City','Country','Lat','Lng', 'Humidity', 'Hotel Name',]
)
# Display the map
# YOUR CODE HERE
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)